In [ ]:
# importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# this function will display shape,head,info,describe,isna

def disply(df):
    display('--------------------------------',df.shape,'------------------------',df.head(),'------------------------',df.info(),'------------------------',df.describe(),'------------------------',df.isna().sum())

In [ ]:
test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
disply(test)

In [ ]:
item_categeries = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
disply(item_categeries)

In [ ]:
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
disply(items)

In [ ]:
sales = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
disply(sales)

In [ ]:
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')
disply(shops)

In [ ]:
# merging Dataset
df1 = pd.merge(items,item_categeries,on='item_category_id',how='inner')
df2 = pd.merge(sales,shops,on='shop_id',how = 'inner')
df = pd.merge(df2,df1,on='item_id',how='inner')

In [ ]:
disply(df)

In [ ]:
# now we will keep the data which is also present in test data
df = df[df['shop_id'].isin(test['shop_id'].unique())]
df = df[df['item_id'].isin(test['item_id'].unique())]

In [ ]:
df.head()

In [ ]:
# ploting  unique values of every feature:
def unique_p(df):
    l1 = [i for i in df.columns]
    l2 = [df[i].nunique() for i in df.columns]
    l1 = pd.DataFrame(l1)
    l2 = pd.DataFrame(l2)
    df2 = pd.concat([l1,l2],axis=1)
    df2.columns = ['Names','Unique_values']
    plt.figure(figsize=(18,7))
    g = sns.barplot(x = 'Names',
            y = 'Unique_values',
            data = df2)
    for index, row in df2.iterrows():
        g.text(row.name,row.Unique_values,round(row.Unique_values,2),color='black', ha="center")

In [ ]:
unique_p(df)

In [ ]:
# now we will feature engnr  date column and break it into day and month , 
# so we can extract more information with respect to time

df['date'] = pd.to_datetime(df['date'])
df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year


In [ ]:
df

In [ ]:
# Checking diffrent no of years present in our Df
l = df['date'].dt.year.unique()
print(f'NO OF YEARS {l}')

In [ ]:
# there are diffrent 3 years availible 
# so we will seprately explore each year seprately

# seprating years

#2013
df_2013 = (df[df['date'].dt.year == 2013]).drop('date',axis = 1)
#2014
df_2014 = (df[df['date'].dt.year == 2014].reset_index()).drop('date',axis = 1)

#2015
df_2015 = (df[df['date'].dt.year == 2015].reset_index()).drop('date',axis = 1)


In [ ]:
# now we will analyse every year seprately

In [ ]:
# plotting unique values of 2013
unique_p(df_2013)

In [ ]:
# now we will analyse,
# which is the most expensive product of in every month seprately

# this function will give top expensive products of every months in form of DataFrame
def func1(df):
    new_df = pd.DataFrame()
    for i in range(1,13):
        temp = df[df['month']==i]
        temp.sort_values(by='item_price',inplace = True,ascending = False)
        new_df = pd.concat([temp[:1],new_df])
    return new_df.sort_values(by = 'month')

# sunburst function
def func2(df):
    import plotly.express as px
    fig = px.sunburst(df, path=[i for i in df.columns], values='month')
    fig.show()

In [ ]:
# WE analysed all top expensive products of every month  with help of sunburst plot
temp1 = func1(df_2013)
func2(temp1)

In [ ]:
# now we will analyse in which month people spend more money

# this function will give two list x,y 
def func2(df):
    temp2 = func1(df)
    plt.rcParams['figure.figsize'] = [8, 8]
    sns.barplot(data=temp2, x="month", y="item_price")
    plt.show()

In [ ]:
func2(df_2013)

In [ ]:
# now we will see, what kind of item category is famous among the customer with help of pie chart we will visualise 


# this function give customised dataframe for a particular month sorted by feat
def func3(df,m,y,feat):
    from calendar import monthrange
    df1 = df[df['month']==m]
    d = list(monthrange(y, m))[1]
    for i in  range(1,d+1):
        df2 = df1[df['day']==i]
        df2.sort_values(by=feat,inplace = True,ascending = False)
    return df2

# this function creates a dataframe of percent of count in context with feat1 
def func4(df,feat1):
    lis = list(df[feat1])
    l = len(lis)
    values = []
    for i in lis:
        c = lis.count(i)
        values.append(round((c/l)*100,1))
    names  = lis 
    df2 = pd.DataFrame(list(zip(names,values)),columns =['names', 'values'])
    return df2


# this function plots piechart
def func5(df2,m,feat2):
    l = df2['names'].nunique()
    fig = px.pie(df2, values='values', names='names')
    fig.update_traces(textposition='inside', textinfo='percent+label')
    fig.update_layout(title = dict(text = f'{m} month - {feat2} total values : {l}'),)
    fig.show()

In [ ]:
for i in range(1,13):
    temp1 = func3(df_2013,1,2013,'item_price')
    temp2 = func4(temp1,'item_category_name')
    func5(temp2,i,'item_category_name')

In [ ]:
# now we will visualise favourite shops of customers with help of chart 
# shop id and shop_name are basically same so we will use shop_name
# and during time of model makiing we will drop one of them 

for i in range(1,13):
    temp1 = func3(df_2013,1,2013,'item_price')
    temp2 = func4(temp1,'shop_name')
    func5(temp2,i,'shop_name')

In [ ]:
# now we will do same for 2014

In [ ]:
unique_p(df_2014)

In [ ]:
temp1 = func1(df_2014)
func2(temp1)

In [ ]:
func2(df_2014)

In [ ]:
for i in range(1,13):
    temp1 = func3(df_2014,1,2014,'item_price')
    temp2 = func4(temp1,'item_category_name')
    func5(temp2,i,'item_category_name')

In [ ]:
for i in range(1,13):
    temp1 = func3(df_2014,1,2014,'item_price')
    temp2 = func4(temp1,'shop_name')
    func5(temp2,i,'shop_name')

In [ ]:
# now we will do same for 2015

In [ ]:
unique_p(df_2015)

In [ ]:
temp1 = func1(df_2015)
func2(temp1)

In [ ]:
func2(df_2015)

In [ ]:
for i in range(1,13):
    temp1 = func3(df_2015,1,2015,'item_price')
    temp2 = func4(temp1,'item_category_name')
    func5(temp2,i,'item_category_name')

In [ ]:
for i in range(1,13):
    temp1 = func3(df_2015,1,2015,'item_price')
    temp2 = func4(temp1,'shop_name')
    func5(temp2,i,'shop_name')

In [ ]:
# now i guess its enough for basic visualisation
# now we will do some feature engineering

In [ ]:
# droping irrelevant features
df = df.drop(['date', 'date_block_num','shop_name', 'item_name','item_category_name'],axis = 1)

In [ ]:
# heatmap
sns.heatmap(df.corr(),linewidths=0.5,linecolor='white',annot=True)

In [ ]:
x = df[['shop_id', 'item_id', 'item_cnt_day', 'item_category_id','day', 'month','year']]
y = df['item_price']

In [ ]:
# now we will split the data
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,shuffle = False,test_size = 0.3)

In [ ]:
# now we will import linear regression
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x_train,y_train)

In [ ]:
# Now  we will evaluate the model 
from sklearn.metrics import mean_squared_error
y_pred = model.predict(x_test)
mean_squared_error(y_test,y_pred,squared=False)

In [ ]:
# we will not do any kind hyperparameter tunning or feature engr to increase the accuracy
# because it is just an example of simple linear regression